In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table, ctx
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# Import the CRUD library
from AnimalShelterDriver import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "Yptz693"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned

# This inital read is not necessary
# During the inital creation of the components the callback for the components is triggered. 
# Since no options are selected during this time it falls to the last case which is to read the entire database.

df = pd.DataFrame.from_records(db.read({}))  #redundent and cause 2 inital reads on startup

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)

# Our CRUD library drops the '_id ' column'
if "_id" in df.columns:   # Test if '_id' column exists, if so remove it
     df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Place the HTML image tag in the line below into the app.layout code according to your design
# Also remember to include a unique identifier such as your name or date


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(className='row',style={'display' : 'flex'},
        children=[
    html.Img(style={'width': '250px', 'height': '250px'},src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('CS-340 Dashboard - David S. Harmor')))]),
    html.Hr(),
    html.Div(className='buttonRow',   # Add are Tab buttons
            style={'display' : 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, children='Cats'),
                    html.Button(id='submit-button-two', n_clicks=0, children='Dogs')
                ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True,
             
            } for i in df.columns
        ],
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10        
        
    ),   
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(        className='row',
         style={'display' : 'flex'},
             children=[                 
                 dcc.RadioItems(
            id='graph-type',
            options=[
                {'label': 'Histogram', 'value': 'Hist'},
                {'label': 'Pie-Chart', 'value': 'Pie'},
         
            ],
            value='Hist'
          ),     
        html.Div(
            id='graph-id',
            className='col s12 m6'
            ),
            html.Div(
            id='map-id',
            className='col s12 m6'
            ),
                 html.H1("Hello World!!!")
            ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################
    
#@app.callback(Output('datatable-id','data'),
#              [Input('filter-type', 'value')])
#def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
#  return
# Radio button filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value'),
               Input('submit-button-one', 'n_clicks'),
               Input('submit-button-two', 'n_clicks')
               ],
              prevent_initial_call=True
             )
def update_dashboard(filter_type,btn1,btn2):    
       
        if ctx.triggered_id == "filter-type":
           
            # Water Rescue filter
            if filter_type == 'water':
                df = pd.DataFrame.from_records(db.read({
                    "animal_type": "Dog",
                    "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                     ]},
                    "sex_upon_outcome": "Intact Female",
                    "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
                }))
            # Mountain or Wilderness Rescue filter
            elif filter_type == 'mount':
                df = pd.DataFrame.from_records(db.read({
                    "animal_type": "Dog",
                    "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                      "Siberian Husky", "Rottweiler"
                                     ]},
                    "sex_upon_outcome": "Intact Male",
                    "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
                }))

            # Disaster Rescue or Individual Tracking filter
            elif filter_type == 'disaster':
                df = pd.DataFrame.from_records(db.read({
                    "animal_type": "Dog",
                    "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                      "Bloodhound","Rottweiler"
                                     ]},
                    "sex_upon_outcome": "Intact Male",
                    "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
                }))
            # Reset to no filter
            elif filter_type == 'reset':
                df = pd.DataFrame.from_records(db.read({}))
            # Determine which button was selected by object name
            # this prevents unnecessary calls to the database and requires 1 click by the user
        elif "submit-button-one" == ctx.triggered_id: # Determine if "Cats" was selected
              df = pd.DataFrame.from_records(db.read({"animal_type" : "Cat"}))        
        elif "submit-button-two" == ctx.triggered_id: # Determine if "Dogs" was selected
              df = pd.DataFrame.from_records(db.read({"animal_type" : "Dog"}))    
        else:
              df = pd.DataFrame.from_records(db.read({}))
    
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
            
     # Cleanup Mongo _id field
    
    # Our CRUD library drops the '_id ' column'    
        if "_id" in df.columns:    # Test if '_id' column exists, if so remove it
             df.drop(columns=['_id'],inplace=True)
        
        return (data,columns)

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),    
     Input('graph-type','value')
    ]) # derived_viewport_data,graph_type)
def update_graphs(viewData,graph_type):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #    
    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    
    if viewData is None:
        return    
    
    df = pd.DataFrame.from_dict(viewData)   
    if graph_type == "Pie":
        return [dcc.Graph(            
                figure = px.pie(df, names='breed', title='Preferred Animals'))  
            ]
    else:
        return [dcc.Graph(            
                figure = px.histogram(df,'breed')) # Create Histogram                
            ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{'if': { 'column_id': i },'background_color': '#D2F3FF'} for i in selected_columns]



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
   
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '500px', 'height': '500px'}, center=[dff.iloc[row,13],dff.iloc[row,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:13155/
